In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity
import random

# EDA & Feature Augmentation

After we were suggested by teacher to do a recommendation system by content based.As for the dataset that we had before did not have enought features to do in content based way. We decided to scrape more data from the website.

We scrape movie's id by using API and with this ID it doesn't have any affect on the recommendation. We just need the id for fetching poster.

In [ ]:
import requests
import csv

base_url = "https://api.themoviedb.org/3/movie/top_rated"
api_key = "a7cdaf4cd0359210b90afdd4fe28b356"
language = "en-US"
page = 1
total_movies = 0

csv_file = "movie_data.csv"
csv_header = ["ID", "Title"]

with open(csv_file, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(csv_header)

    while total_movies < 9500:
        url = f"{base_url}?api_key={api_key}&language={language}&page={page}"
        response = requests.get(url)
        data = response.json()

        if 'results' in data:
            results = data['results']
            if len(results) == 0:
                break  # No more results, exit the loop

            for result in results:
                movie_id = result['id']
                title = result['original_title']
                writer.writerow([movie_id, title])
                total_movies += 1
                if total_movies >= 9500:
                    break  # Reached the desired number of movie IDs

        page += 1

print("Scraping and saving to CSV completed.")


In [ ]:
movies_names =[]
directors = []
stars = []
overviews = []
genres = []

In [ ]:
pages = np.arange(1,9500,50)

In [ ]:
for page in pages:
    page = requests.get("https://www.imdb.com/search/title/?title_type=feature&num_votes=10000,&sort=user_rating,desc&start="+str(page)+"&ref_=adv_nxt")
    soup = BeautifulSoup(page.text,'lxml')
    movie_data = soup.find_all('div', class_="lister-item mode-advanced")
    for store in movie_data:

        name = store.h3.a.text
        movies_names.append(name)
        overview = store.find("div", class_="lister-item-content").find_all('p')[1].text.strip().replace("See full summary\xa0»","")
        overviews.append(overview)

        director = store.find("div", class_="lister-item-content").find("p", class_="").find("a").text
        directors.append(director)

        genre_element = store.find("span", class_="genre")
        genre = genre_element.text.strip() if genre_element else ""
        genres.append(genre)

        stars_element = store.find("p", class_="").find_all("a")
        star1 = ", ".join(star.text.strip() for star in stars_element) 
        stars_list = star1.split(', ')
        stars_list = stars_list[1:]
        stars_string = ", ".join(stars_list)
        stars.append(stars_string)
        

      
        

In [ ]:
added_variable = pd.DataFrame({"MovieName": movies_names,"Genre": genres, "Directors": directors, "Stars": stars,"Overviews": overviews})

In [ ]:
added_variable.to_csv('new_dataset.csv',index=False)

In [ ]:
df = pd.read_csv('new_dataset.csv')

In [ ]:
df.duplicated().sum()

0

After that we merged the old dataset with the new one.

In [ ]:
new_dataset = pd.read_csv("new_dataset.csv")

old_dataset = pd.read_csv("encoded_clean_1.csv")

# Merge the datasets based on the movie names
df_merged = pd.merge(new_dataset, old_dataset, left_on="MovieName", right_on="Name_of_Movie")

df_aligned = df_merged[["MovieName", "Genre", "Directors","Overviews","Stars","Watch_time","Year_of_Release","Meta_Score","Votes","Gross_Collection(M$)"]]



In [ ]:
df_aligned

In [ ]:
df_aligned.to_csv('Merged_df.csv',index=False)

In [ ]:
df_aligned.duplicated().sum()

0

# Start running the code from here

# <u><b><font color='yellow'>Content Based Recommender</font></b></u>
The Content Based Recommender offers generalized recommendations to user based on a particular item. This system uses item metadata such as :
- Movie Star, Director, Overview, Genre,etc.

For movies, to make these recommendation. The general idea behind these recommender systems is that if a person liked a particular item, he or she will also like an item that is similar to it.

#### Loading Merge Data

In [2]:
movies = pd.read_csv('Final_Merged.csv')

In [3]:
movies.head()

,id,MovieName,Genre,Directors,Stars,Overviews,Watch_time,Year_of_Release,Meta_Score,Votes,Gross_Collection(M$)
0,278.0,The Shawshank Redemption,Drama,Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...","Over the course of several years, two convicts...",142,1994,82,2728085,28.34
1,238.0,The Godfather,"Crime, Drama",Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Diane Ke...","Don Vito Corleone, head of a mafia family, dec...",175,1972,100,1896661,134.97
2,NaN,Ramayana: The Legend of Prince Rama,"Animation, Action, Adventure",Ram Mohan,"Yûgô Sakô, Koichi Saski, Arun Govil, Nikhil Ka...",An anime adaptation of the Hindu epic the Rama...,87,1975,59,47591,16.94
3,NaN,Hababam Sinifi,"Comedy, Drama",Ertem Egilmez,"Kemal Sunal, Münir Özkul, Halit Akçatepe, Tari...","Lazy, uneducated students share a very close b...",152,2008,84,2701057,534.86
4,155.0,The Dark Knight,"Action, Crime, Drama",Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",When the menace known as the Joker wreaks havo...,195,1993,95,1377562,96.90


In [4]:
movies.duplicated().sum()

0

In [5]:
movies.isna().sum()

id                      3659
MovieName                  0
Genre                      0
Directors                  0
Stars                      0
Overviews                  0
Watch_time                 0
Year_of_Release            0
Meta_Score                 0
Votes                      0
Gross_Collection(M$)       0
dtype: int64

In [6]:
random_numbers = [random.choice([537061,537061,475557,678580,615457]) for _ in range(len(movies))]

movies['id'] = movies['id'].fillna(pd.Series(random_numbers, index=movies.index))

movies['id'] = movies['id'].astype(int)


In [7]:
movies.iloc[0]['Overviews']

'Over the course of several years, two convicts form a friendship, seeking consolation and, eventually, redemption through basic compassion.'

In [8]:
#Tim Robbins

#TimRobbins

def remove_spaces(text):
    return text.replace(" ", "")

movies['Stars'] = movies['Stars'].apply(remove_spaces)

#### We're converting every columns to list so we concatenate them into one column known as **tags**.

In [9]:
def convert_int_columns_to_string(movies, column_names):
    for column in column_names:
        movies[column] = movies[column].apply(lambda x: str(x))
    return movies
columns_to_convert = ['Watch_time','Year_of_Release','Meta_Score','Votes','Gross_Collection(M$)']
convert_int_columns_to_string(movies, columns_to_convert)


,id,MovieName,Genre,Directors,Stars,Overviews,Watch_time,Year_of_Release,Meta_Score,Votes,Gross_Collection(M$)
0,278,The Shawshank Redemption,Drama,Frank Darabont,"TimRobbins,MorganFreeman,BobGunton,WilliamSadler","Over the course of several years, two convicts...",142,1994,82,2728085,28.34
1,238,The Godfather,"Crime, Drama",Francis Ford Coppola,"MarlonBrando,AlPacino,JamesCaan,DianeKeaton","Don Vito Corleone, head of a mafia family, dec...",175,1972,100,1896661,134.97
2,537061,Ramayana: The Legend of Prince Rama,"Animation, Action, Adventure",Ram Mohan,"YûgôSakô,KoichiSaski,ArunGovil,NikhilKapoor,Ed...",An anime adaptation of the Hindu epic the Rama...,87,1975,59,47591,16.94
3,537061,Hababam Sinifi,"Comedy, Drama",Ertem Egilmez,"KemalSunal,MünirÖzkul,HalitAkçatepe,TarikAkan","Lazy, uneducated students share a very close b...",152,2008,84,2701057,534.86
4,155,The Dark Knight,"Action, Crime, Drama",Christopher Nolan,"ChristianBale,HeathLedger,AaronEckhart,Michael...",When the menace known as the Joker wreaks havo...,195,1993,95,1377562,96.9
...,...,...,...,...,...,...,...,...,...,...,...
9492,676547,Visions,"Horror, Mystery",Kevin Greutert,"IslaFisher,AnsonMount,GillianJacobs,JoannaCassidy",After moving to a vineyard to start a new life...,88,2021,32,47591,16.94
9493,340102,Housefull 2,"Action, Comedy, Drama",Sajid Khan,"AkshayKumar,JohnAbraham,AsinThottumkal,Jacquel...",A comedy of errors wherein four men help each ...,88,2021,32,47591,16.94
9494,537061,Celtic Pride,"Comedy, Sport",Tom DeCerchio,"DamonWayans,DanielStern,DanAykroyd,GailO'Grady",Two over-loyal Celtic fans kidnap their oppone...,111,2020,31,47591,16.94
9495,537061,Wrecked,"Adventure, Drama, Mystery",Michael Greenspan,"AdrienBrody,CarolineDhavernas,RyanRobbins,Adri...",A man wakes in a car wreck at the bottom of a ...,115,2022,53,47591,16.94


In [10]:
def convert_string_columns_to_list(movies, column_names):
    for column in column_names:
        movies[column] = movies[column].apply(lambda x: x.split(','))
    return movies

columns_to_convert = ['Genre', 'Directors','Stars','Watch_time','Year_of_Release','Meta_Score','Votes','Gross_Collection(M$)','Overviews']
convert_string_columns_to_list(movies, columns_to_convert)

,id,MovieName,Genre,Directors,Stars,Overviews,Watch_time,Year_of_Release,Meta_Score,Votes,Gross_Collection(M$)
0,278,The Shawshank Redemption,[Drama],[Frank Darabont],"[TimRobbins, MorganFreeman, BobGunton, William...","[Over the course of several years, two convic...",[142],[1994],[82],[2728085],[28.34]
1,238,The Godfather,"[Crime, Drama]",[Francis Ford Coppola],"[MarlonBrando, AlPacino, JamesCaan, DianeKeaton]","[Don Vito Corleone, head of a mafia family, ...",[175],[1972],[100],[1896661],[134.97]
2,537061,Ramayana: The Legend of Prince Rama,"[Animation, Action, Adventure]",[Ram Mohan],"[YûgôSakô, KoichiSaski, ArunGovil, NikhilKapoo...",[An anime adaptation of the Hindu epic the Ram...,[87],[1975],[59],[47591],[16.94]
3,537061,Hababam Sinifi,"[Comedy, Drama]",[Ertem Egilmez],"[KemalSunal, MünirÖzkul, HalitAkçatepe, TarikA...","[Lazy, uneducated students share a very close...",[152],[2008],[84],[2701057],[534.86]
4,155,The Dark Knight,"[Action, Crime, Drama]",[Christopher Nolan],"[ChristianBale, HeathLedger, AaronEckhart, Mic...",[When the menace known as the Joker wreaks hav...,[195],[1993],[95],[1377562],[96.9]
...,...,...,...,...,...,...,...,...,...,...,...
9492,676547,Visions,"[Horror, Mystery]",[Kevin Greutert],"[IslaFisher, AnsonMount, GillianJacobs, Joanna...",[After moving to a vineyard to start a new lif...,[88],[2021],[32],[47591],[16.94]
9493,340102,Housefull 2,"[Action, Comedy, Drama]",[Sajid Khan],"[AkshayKumar, JohnAbraham, AsinThottumkal, Jac...",[A comedy of errors wherein four men help each...,[88],[2021],[32],[47591],[16.94]
9494,537061,Celtic Pride,"[Comedy, Sport]",[Tom DeCerchio],"[DamonWayans, DanielStern, DanAykroyd, GailO'G...",[Two over-loyal Celtic fans kidnap their oppon...,[111],[2020],[31],[47591],[16.94]
9495,537061,Wrecked,"[Adventure, Drama, Mystery]",[Michael Greenspan],"[AdrienBrody, CarolineDhavernas, RyanRobbins, ...",[A man wakes in a car wreck at the bottom of a...,[115],[2022],[53],[47591],[16.94]


In [11]:
movies['tags'] =movies['Overviews'] + movies['Directors'] + movies['Genre']+ movies['Stars'] + movies['Watch_time']+ movies['Year_of_Release']+ movies['Meta_Score']+ movies['Votes']+movies['Gross_Collection(M$)']

In [12]:
movies.head()

,id,MovieName,Genre,Directors,Stars,Overviews,Watch_time,Year_of_Release,Meta_Score,Votes,Gross_Collection(M$),tags
0,278,The Shawshank Redemption,[Drama],[Frank Darabont],"[TimRobbins, MorganFreeman, BobGunton, William...","[Over the course of several years, two convic...",[142],[1994],[82],[2728085],[28.34],"[Over the course of several years, two convic..."
1,238,The Godfather,"[Crime, Drama]",[Francis Ford Coppola],"[MarlonBrando, AlPacino, JamesCaan, DianeKeaton]","[Don Vito Corleone, head of a mafia family, ...",[175],[1972],[100],[1896661],[134.97],"[Don Vito Corleone, head of a mafia family, ..."
2,537061,Ramayana: The Legend of Prince Rama,"[Animation, Action, Adventure]",[Ram Mohan],"[YûgôSakô, KoichiSaski, ArunGovil, NikhilKapoo...",[An anime adaptation of the Hindu epic the Ram...,[87],[1975],[59],[47591],[16.94],[An anime adaptation of the Hindu epic the Ram...
3,537061,Hababam Sinifi,"[Comedy, Drama]",[Ertem Egilmez],"[KemalSunal, MünirÖzkul, HalitAkçatepe, TarikA...","[Lazy, uneducated students share a very close...",[152],[2008],[84],[2701057],[534.86],"[Lazy, uneducated students share a very close..."
4,155,The Dark Knight,"[Action, Crime, Drama]",[Christopher Nolan],"[ChristianBale, HeathLedger, AaronEckhart, Mic...",[When the menace known as the Joker wreaks hav...,[195],[1993],[95],[1377562],[96.9],[When the menace known as the Joker wreaks hav...


In [13]:
new_df = movies[['id','MovieName','tags']]

In [14]:
new_df.head()

,id,MovieName,tags
0,278,The Shawshank Redemption,"[Over the course of several years, two convic..."
1,238,The Godfather,"[Don Vito Corleone, head of a mafia family, ..."
2,537061,Ramayana: The Legend of Prince Rama,[An anime adaptation of the Hindu epic the Ram...
3,537061,Hababam Sinifi,"[Lazy, uneducated students share a very close..."
4,155,The Dark Knight,[When the menace known as the Joker wreaks hav...


### We're converting the **Tag** column into String Type So we're able to apply Bag of Word technique.

In [15]:
new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))

C:\Users\ROG Zephyrus PC\AppData\Local\Temp\ipykernel_34796\3089450492.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))


In [16]:
new_df.iloc[0]['tags']

'Over the course of several years  two convicts form a friendship  seeking consolation and  eventually  redemption through basic compassion. Frank Darabont Drama TimRobbins MorganFreeman BobGunton WilliamSadler 142 1994 82 2728085 28.34'

In [17]:
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())
new_df.head()

C:\Users\ROG Zephyrus PC\AppData\Local\Temp\ipykernel_34796\842480936.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())


,id,MovieName,tags
0,278,The Shawshank Redemption,over the course of several years two convicts...
1,238,The Godfather,don vito corleone head of a mafia family dec...
2,537061,Ramayana: The Legend of Prince Rama,an anime adaptation of the hindu epic the rama...
3,537061,Hababam Sinifi,lazy uneducated students share a very close b...
4,155,The Dark Knight,when the menace known as the joker wreaks havo...


### Applying **PorterStemmer** on **Tags**

This is an optional step but performing stemming using Porter Stemmer algorithm can be be benificial.
Stemming is a technique used to reduce words to their base or root form, allowing for better grouping and normalization of words with similar meanings.

Example:['Loving','Loved','Lovely']

After applying Porter Stemmer: ['Love','Love','Love']        

In [18]:
import nltk
from nltk.stem.porter import PorterStemmer

In [19]:
ps = PorterStemmer()

In [20]:
def stem(text):
    y = []

    for i in text.split():
        y.append(ps.stem(i))
    
    return " ".join(y)

In [21]:
new_df['tags'] = new_df['tags'].apply(stem)

C:\Users\ROG Zephyrus PC\AppData\Local\Temp\ipykernel_34796\3213734980.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(stem)


In [22]:
new_df['tags'][1]

'don vito corleon head of a mafia famili decid to hand over hi empir to hi youngest son michael. howev hi decis unintent put the live of hi love one in grave danger. franci ford coppola crime drama marlonbrando alpacino jamescaan dianekeaton 175 1972 100 1896661 134.97'

# **<font color='yellow'>Performing Bag of Word Technique </font>** 

The Bag of Words is a representation technique commonly used in natural language processing (NLP) to transform text data into a numerical format that machine learning algorithms can understand. It simplifies a document by considering it as an unordered collection or "bag" of words, without considering grammar or word order. The BoW model focuses solely on the occurrence and frequency of words in the document.

**Example**: 

**The sentence** 

"Squealing suitcase squids are not like regular squids." could be changed into the following BoW dictionary:

{'squeal': 1, 'like': 1, 'not': 1, 'suitcase': 1, 'be': 1, 'regular': 1, 'squid': 2}

#### **<font color='yellow'>CountVectorizer </font>** 

CountVectorizer is a feature extraction technique provided by the scikit-learn library in Python. 

It is specifically designed for converting text documents into numerical feature vectors. CountVectorizer transforms a collection of text documents into a matrix where each row represents a document, and each column represents a unique word

The value in each cell of the matrix represents the count or frequency of the corresponding word in the document.

In [23]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=40000,stop_words='english') 

In [24]:
vectors = cv.fit_transform(new_df['tags'])

In [25]:
vectors.shape

(9497, 40000)

In [26]:
cv.get_feature_names_out()

array(['00', '000', '007', ..., 'øyvindhagen', 'ümmüputgül', 'ünlü'],
      dtype=object)

# **<font color='yellow'>Model</font>**

### **Performing KNN (K-Nearest Neighbors) with Cosine Similarity**

## Cosine Similarity

Cosine similarity is a metric used to measure the similarity between two vectors by computing the cosine of the angle between them. 

In [27]:
 # Number of neighbors to consider
k = 10
knn_model = NearestNeighbors(n_neighbors=k, metric='cosine')
knn_model.fit(vectors)
distances, indices = knn_model.kneighbors(vectors)


In [28]:
def recommend(movie):
    movie_index = new_df[new_df['MovieName'] == movie].index[0]
    target_movie = vectors[movie_index]

    _, top_movie_indices = knn_model.kneighbors(target_movie)
    distances,_= knn_model.kneighbors(target_movie)

    ## Flatten the distance array andtarget_movie = vectors[movie_index] top_movie_indices array
    distances = distances.flatten()
    top_movie_indices = top_movie_indices.flatten()
    top_similar_movies = new_df.iloc[top_movie_indices]['MovieName']

    for i in top_similar_movies:
        print(i)
    


In [29]:
recommend('Thor')

Thor
Thor: The Dark World
Thor: Love and Thunder
Thor: Ragnarok
Hannibal
Bright
Power Rangers
Paranormal Activity
Miami Vice
Eternals


### Saving the result file into Pickle File

In [30]:
import pickle

In [31]:
pickle.dump(new_df,open('movielist.pkl','wb'))

In [32]:
pickle.dump(distances,open('similarity.pkl','wb'))

In [33]:
pickle.load(open('similarity.pkl','rb'))

array([[0.00000000e+00, 8.40000000e-01, 8.43107092e-01, ...,
        8.51443729e-01, 8.53940651e-01, 8.60737875e-01],
       [0.00000000e+00, 4.57884801e-01, 5.24443646e-01, ...,
        5.95854812e-01, 5.98336791e-01, 5.99444830e-01],
       [2.22044605e-16, 6.25497057e-01, 6.76710456e-01, ...,
        7.06705770e-01, 7.08641727e-01, 7.09677419e-01],
       ...,
       [4.44089210e-16, 7.09828354e-01, 7.19775731e-01, ...,
        7.54641754e-01, 7.57318664e-01, 7.61023940e-01],
       [0.00000000e+00, 7.08079820e-01, 7.14226197e-01, ...,
        7.38138532e-01, 7.40026527e-01, 7.40026527e-01],
       [2.22044605e-16, 5.70227753e-01, 5.99952654e-01, ...,
        6.58838477e-01, 6.61130503e-01, 6.61498400e-01]])

In [34]:
pickle.dump(vectors,open('vectors.pkl','wb'))

In [35]:
new_df['id'] = new_df['id'].astype(int)

C:\Users\ROG Zephyrus PC\AppData\Local\Temp\ipykernel_34796\1403378.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['id'] = new_df['id'].astype(int)


In [36]:
pickle.load(open('movielist.pkl','rb'))

,id,MovieName,tags
0,278,The Shawshank Redemption,over the cours of sever year two convict form ...
1,238,The Godfather,don vito corleon head of a mafia famili decid ...
2,537061,Ramayana: The Legend of Prince Rama,an anim adapt of the hindu epic the ramayana w...
3,537061,Hababam Sinifi,lazi uneduc student share a veri close bond. t...
4,155,The Dark Knight,when the menac known as the joker wreak havoc ...
...,...,...,...
9492,676547,Visions,after move to a vineyard to start a new life w...
9493,340102,Housefull 2,a comedi of error wherein four men help each o...
9494,537061,Celtic Pride,two over-loy celtic fan kidnap their opponent'...
9495,537061,Wrecked,a man wake in a car wreck at the bottom of a r...
